# Introduction to Creating Tables

To try out how to create tables and insert data into the tables we are using [duckdb](https://duckdb.org/) in this notebook. It is creating a local database which you can run SQL-queries in Jupyter Notebooks against. DBeaver also has a connector so you can also run SQL-queries in there. Duckdb works with [sqlalchemy](https://www.sqlalchemy.org/) a widely used python library to connect to databases and ipython-sql. Ipython-sql allows us to convert a Jupyter code cell into a SQL cell with the jupyter magic command ```%%sql```.

In this notebook you will learn how to create tables and insert data into the created tables.

In [1]:

import duckdb
import pandas as pd
import sqlalchemy
import json
# No need to import duckdb_engine
# SQLAlchemy will auto-detect the driver needed bASed on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

Set configrations on ipython-sql to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connect ipython-sql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [3]:
%sql duckdb:///sampleDB.duckdb

We have two csv files in our data directory:
- raw_orders.csv
- raw_payments.csv

and a json file:
- raw_customers.json


So we will see how to insert csv files and json files into our database but first we will create a tables for the data and for that let's have a look at the data and headers.

In [4]:
df_customers = pd.read_json('data/raw_customers.json', orient='records', lines=True)
df_customers.head()

,id,first_name,last_name
0,1,Michael,P.
1,2,Shawn,M.
2,3,Kathleen,P.
3,4,Jimmy,C.
4,5,Katherine,R.


In [5]:
df_orders = pd.read_csv('data/raw_orders.csv')
df_orders.head()

,id,user_id,order_date,status
0,1,1,2018-01-01,returned
1,2,3,2018-01-02,completed
2,3,94,2018-01-04,completed
3,4,50,2018-01-05,completed
4,5,64,2018-01-05,completed


In [6]:
df_payment = pd.read_csv('data/raw_payments.csv')
df_payment.head()

,id,order_id,payment_method,amount
0,1,1,credit_card,1000
1,2,2,credit_card,2000
2,3,3,coupon,100
3,4,4,coupon,2500
4,5,5,bank_transfer,1700


Let us start with creating the tables, for that it is standard to define the column types. The column types can differ from sql flavour to sql flavour.

We will start with the ```raw_cusomer``` table here we have:
- id
- first_name
- last_name

We will also define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The other two columns are strings which is call ```VARCHAR``` in sql.

In [7]:
%%sql

CREATE TABLE raw_customers(
    id INTEGER PRIMARY KEY NOT NULL,
    first_name VARCHAR,
    last_name VARCHAR
    );


,Success


Next we will create the ```raw_orders``` table:
- id
- user_id
- order_date
- status

We will define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The ```user_id``` is a ```FOREIGN KEY``` and references the raw_customer id. Order_date is a ```DATE``` and status is a ```VARCHAR```.

In [8]:
%%sql

CREATE TABLE raw_orders(
    id INTEGER PRIMARY KEY NOT NULL,
    user_id INTEGER,
    FOREIGN KEY(user_id) REFERENCES raw_customers(id),
    order_date DATE,
    status VARCHAR
    );

,Success


Last we will create the ```raw_payments``` table:
- id
- order_id
- payment_method
- amount

Again we will define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The ```order_id``` is a ```FOREIGN KEY``` and references the raw_orders id. payment_method is a ```VARCHAR``` and amount is a ```INTEGER```.

In [9]:
%%sql

CREATE TABLE raw_payments(
    id INTEGER PRIMARY KEY NOT NULL,
    order_id INTEGER,
    FOREIGN KEY(order_id) REFERENCES raw_orders(id),
    payment_method VARCHAR,
    amount INTEGER
    );

,Success


So now we created the tables but at the moment they are empty. Let's check that.

In [10]:
%%sql

select * from raw_customers

,id,first_name,last_name


In [11]:
%%sql

select * from raw_orders

,id,user_id,order_date,status


In [12]:
%%sql

select * from raw_payments

,id,order_id,payment_method,amount


As expected they are empty, so let's change that and use ```INSERT``` for that. Most sql flavours have build in functions to deal with external data sources like csv or json files. In duckdb we can simply use ```read_csv_auto``` to read our csv files and we can than run sql queries against that. For json it is a little bit more complicated we have to create first a table where we store the json itself.

For json we we have to install and load the ```json``` extension from duckdb.

In [13]:
%%sql

INSTALL 'json';
LOAD 'json';

,Success


Now we have to create the table we store the json in. This table has only one column ```j``` with the data type ```JSON```.

In [14]:
%%sql

CREATE TABLE raw_customers_json(
    j JSON
    );


,Success


Now we can use the ```INSERT INTO``` command followed by a select statement where the json is read with ```read_json_objects()```.

In [15]:
%%sql

INSERT INTO raw_customers_json SELECT * FROM read_json_objects('data/raw_customers.json', format='auto')

,Success


Let's have a look if that works:

In [16]:
%%sql

SELECT * FROM raw_customers_json

,j
0,"{""id"":1,""first_name"":""Michael"",""last_name"":""P.""}"
1,"{""id"":2,""first_name"":""Shawn"",""last_name"":""M.""}"
2,"{""id"":3,""first_name"":""Kathleen"",""last_name"":""P.""}"
3,"{""id"":4,""first_name"":""Jimmy"",""last_name"":""C.""}"
4,"{""id"":5,""first_name"":""Katherine"",""last_name"":""..."
...,...
95,"{""id"":96,""first_name"":""Jacqueline"",""last_name""..."
96,"{""id"":97,""first_name"":""Shirley"",""last_name"":""D.""}"
97,"{""id"":98,""first_name"":""Nicole"",""last_name"":""M.""}"
98,"{""id"":99,""first_name"":""Mary"",""last_name"":""G.""}"


Now that we have the json in a database we can parse it and put it into our ```raw_customer``` table, we can parse the json with the ```json_extract``` function with the json and the json_path as input e.g. ```json_extract(j, '$.id')```.
```json
{"id":1,"first_name":"Michael","last_name":"P."}
```
The example above will get the ```id``` 1.

We will also use the ```CAST``` function which converts a value (of any type) into a specified datatype. This will be included in a ```INSERT INTO``` and ```SELECT``` statement. And because there are quotationmarks around the names we ```REPLACE``` them.

In [17]:
%%sql
INSERT INTO raw_customers
SELECT 
    CAST(json_extract(j, '$.id') AS INTEGER) AS id,
    REPLACE(CAST(json_extract(j, '$.first_name')AS VARCHAR),'"','') AS first_name, 
    REPLACE(CAST(json_extract(j, '$.last_name')AS VARCHAR),'"','') AS last_name 
FROM raw_customers_json;

,Success


Inserting the data from the csv files is a little bit easier with the ```read_csv_auto()``` function. 

First for the orders:

In [18]:
%%sql

INSERT INTO raw_orders SELECT id,user_id,order_date,status FROM read_csv_auto('data/raw_orders.csv');

,Success


And for the payments:

In [19]:
%%sql

INSERT INTO raw_payments SELECT id,order_id,payment_method,amount FROM read_csv_auto('data/raw_payments.csv');

,Success


Now let's have a look into the data with SQL:

In [20]:
%%sql

SELECT * FROM raw_customers

,id,first_name,last_name
0,1,Michael,P.
1,2,Shawn,M.
2,3,Kathleen,P.
3,4,Jimmy,C.
4,5,Katherine,R.
...,...,...,...
95,96,Jacqueline,A.
96,97,Shirley,D.
97,98,Nicole,M.
98,99,Mary,G.


In [21]:
%%sql

SELECT * FROM raw_orders

,id,user_id,order_date,status
0,1,1,2018-01-01,returned
1,2,3,2018-01-02,completed
2,3,94,2018-01-04,completed
3,4,50,2018-01-05,completed
4,5,64,2018-01-05,completed
...,...,...,...,...
94,95,27,2018-04-04,placed
95,96,90,2018-04-06,placed
96,97,89,2018-04-07,placed
97,98,41,2018-04-07,placed


In [22]:
%%sql

SELECT * FROM raw_payments

,id,order_id,payment_method,amount
0,1,1,credit_card,1000
1,2,2,credit_card,2000
2,3,3,coupon,100
3,4,4,coupon,2500
4,5,5,bank_transfer,1700
...,...,...,...,...
108,109,95,coupon,2400
109,110,96,gift_card,1700
110,111,97,bank_transfer,1400
111,112,98,bank_transfer,1000


## Exercises

Let's get the RFM features for our data science team.

### 1. Recency

In order to get recency for each customer you need to join the orders and customer table, get the latest order and calculate the recency in days.

In [23]:
%%sql

SELECT c.id, c.first_name, c.last_name, MAX(o.order_date) AS last_order_date, current_date - MAX(o.order_date) AS recency FROM raw_customers AS c
JOIN raw_orders AS o ON c.id = o.user_id
GROUP BY c.id, c.first_name, c.last_name



,id,first_name,last_name,last_order_date,recency
0,3,Kathleen,P.,2018-03-11,2045
1,6,Sarah,R.,2018-02-19,2065
2,8,Frank,R.,2018-03-12,2044
3,9,Jennifer,F.,2018-03-17,2039
4,42,Diana,S.,2018-03-12,2044
...,...,...,...,...,...
57,70,Helen,F.,2018-03-26,2030
58,79,Jack,R.,2018-03-11,2045
59,84,Christina,R.,2018-04-02,2023
60,86,Jason,C.,2018-01-24,2091


### 2. Frequency

For frequency we need to get for each customer the total number of orders

In [24]:
%%sql

SELECT c.id, c.first_name, c.last_name, COUNT(o.id) AS frequency FROM raw_customers AS c
JOIN raw_orders AS o ON c.id = o.user_id
GROUP BY c.id, c.first_name, c.last_name

,id,first_name,last_name,frequency
0,3,Kathleen,P.,3
1,6,Sarah,R.,1
2,8,Frank,R.,2
3,9,Jennifer,F.,1
4,42,Diana,S.,2
...,...,...,...,...
57,70,Helen,F.,2
58,79,Jack,R.,2
59,84,Christina,R.,2
60,86,Jason,C.,1


### 3. Monetary Value

Here you need to also look at payments, and calculate the total money spent by each customer.

In [25]:
%%sql

SELECT c.id, c.first_name, c.last_name, SUM(p.amount) FROM raw_customers AS c
JOIN raw_orders AS o ON c.id = o.user_id
JOIN raw_payments AS p ON o.id = p.order_id
GROUP BY c.id, c.first_name, c.last_name

,id,first_name,last_name,sum(p.amount)
0,3,Kathleen,P.,6500.0
1,54,Rose,M.,5700.0
2,53,Anne,B.,3900.0
3,59,Adam,A.,100.0
4,8,Frank,R.,4500.0
...,...,...,...,...
57,70,Helen,F.,5400.0
58,36,Harold,O.,2800.0
59,47,Marie,P.,3600.0
60,68,Jesse,E.,2300.0


### 4. Bonus questions

Just because we do data engineering or analytics engineering tasks (calculating features), does not mean we shouldn't be curious about the data. 
This curiosity can also help us spot mistakes or data inconsistencies, and become recipes for unit tests.

Here are some questions you can ask.
- when was the last purchase?
- what is the average, median recency? what about the range?
- how much do people spend on average?
- how much did the largest big spender spend?
- what is the share of customers that only buy once?